In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#NER
각 단어가 어떠한 개체명(장소, 시간, 등)을 가르키는지를 확인하는 task  
dataset:https://www.kaggle.com/datasets/debasisdotcom/name-entity-recognition-ner-dataset

##Data PreProcessing  
제공된 data는 이미 tokenization이 되어있는 형태로 제공된다.  
그렇기에 vocab dictionary를 직접 구성하고 각 tokenization된 token들을 indexing하여 숫자형태의 데이터로 변환환다.  
이때 class에 index를 부여해주는 scikit learn의 LabelEncoder를 사용해서 변환환다.  
또한 모든 token의 수를일정하게 맞춰주어야만 tensor 연산이 가능하기 때문에 <PAD>에 대한 indexing도 추가한다.


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch

data_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/5일차/dataset/NER dataset.csv", encoding="ISO-8859-1").loc[:100000,:]
print(data_df)

#PAD를 추가하여 token에 index를 부여하기 위한 label encoder 학습
token_encoder = LabelEncoder()
token_encoder.fit(["<PAD>"]+list(map(lambda x: x.lower(),data_df.loc[:,"Word"])))
token_pad_index = token_encoder.transform(["<PAD>"])
print(token_encoder.transform(["<PAD>"]))

#PAD를 추가하여  label에 index를 부여하기 위한  label encoder 학습
label_encoder = LabelEncoder()
label_encoder.fit(["<PAD>"]+list(data_df.loc[:,"Tag"]))
label_pad_index = label_encoder.transform(["<PAD>"])
print(label_encoder.transform(["<PAD>"]))

data_list = []
label_list = []

for i in range(len(data_df)):
  if type(data_df.loc[i,"Sentence #"]) is str:
    data = []
    label = []
  data.append(data_df.loc[i,"Word"].lower())
  label.append(data_df.loc[i,"Tag"])
  if type(data_df.loc[i,"Sentence #"]) is str:
    data_list.append(data)
    label_list.append(label)

print(data_list)
print(label_list)
print(len(data_list))
print(len(label_list))

print(data_list[0])
print(label_list[0])
print(token_encoder.transform(data_list[0]))
print(label_encoder.transform(label_list[0]))

print(len(data_list[0]))
print(len(label_list[0]))

data_list = list(map(token_encoder.transform, data_list))
label_list = list(map(label_encoder.transform, label_list))

# print(data_list)
# print(label_list)
# print(len(data_list))
# print(len(label_list))


            Sentence #           Word  POS Tag
0          Sentence: 1      Thousands  NNS   O
1                  NaN             of   IN   O
2                  NaN  demonstrators  NNS   O
3                  NaN           have  VBP   O
4                  NaN        marched  VBN   O
...                ...            ...  ...  ..
99996              NaN      seriously   RB   O
99997              NaN              .    .   O
99998   Sentence: 4544  Demonstrators  NNS   O
99999              NaN       chanting  VBG   O
100000             NaN              "   ``   O

[100001 rows x 4 columns]
[487]
[0]
[['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.'], ['families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banners', 'with', 'such', 'slogans', 'as', '"', 'bush', 'numbe

##Dataset  
전체 데이터중 4000개의 data는 train 나머지는 test로 사용한다.  
사전에 tokenization과 indexing된 데이터의 크기를 맞추기위해 data에 padding을 추가한다.  
  padding의 value는 <PAD> token의 index를 사용한다.

In [5]:
import torch

#train_test_split
train_data_list = data_list[:4000]
test_data_list = data_list[4000:]
train_label_list = label_list[:4000]
test_label_list = label_list[4000:]

class myDataset(Dataset):
  def __init__(self,data_list, label_list) -> None:
      super().__init__()
      self.data_list = data_list
      self.label_list = label_list

  def __len__(self):
      return len(self.data_list)

  def __getitem__(self, index):
      #make data with padding
      data_pad = torch.zeros(100,dtype=torch.int64)+token_pad_index   # zeros는 0으로 패딩 채우는거
      label_pad = torch.zeros(100)+label_pad_index

      data = torch.cat([torch.IntTensor(self.data_list[index]),data_pad])[:50]
      label = torch.cat([torch.IntTensor(self.label_list[index]),label_pad])[:50]
      return data.flip(-1), label.flip(-1)

train_dataset = myDataset(train_data_list, train_label_list)
test_dataset = myDataset(test_data_list, test_label_list)

for i in train_dataset:
  print(i)
  break
  
batch_size = 50
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

for i in train_dataloader:
  print(i[0].shape)
  print(i[1].shape)
  break

(tensor([ 487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,
         487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,
         487,  487,   15, 2530, 9163, 3983, 9403, 1687, 6432, 9973, 9165, 2803,
         933, 4970, 4744, 9813, 9165, 7228, 9254, 5570, 9217, 5741, 4409, 2824,
        6432, 9200]), tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 17., 17.,
        17., 17., 17.,  4., 17., 17., 17., 17., 17.,  3., 17., 17., 17., 17.,
        17.,  3., 17., 17., 17., 17., 17., 17.], dtype=torch.float64))
torch.Size([50, 50])
torch.Size([50, 50])


##Model  
lstm을 위한 model과 seq2seq를 이용한 모델 두가지 구성  
seq2seq는 encoder model과 decoder model을 따로 구성하고 encoder model의 h,c를 decoer model의 입력으로 사용한다.  
이후 decoder model의 hidden state 들을 예측을 위해 사용한다.

# 새 섹션

In [13]:
from torch import nn

class myModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()

      self.emb = nn.Embedding(50000, 128)
      self.lstm = nn.LSTM(128,128, batch_first=True)
      self.ln1 = nn.Linear(128,100)
      self.ln2 = nn.Linear(100,20)

  def forward(self, x):
      x = self.emb(x)
      x, (h,c) = self.lstm(x)
      # x.shape : 3, 50, 128    h.shape : 3, 128    c.shape : 3, 128
      x = self.ln1(x)
      x = self.ln2(x)

      return x

model = myModel()

for i in train_dataloader:
  print(i[0].shape)
  print(model(i[0]).shape)
  break

torch.Size([50, 50])


AttributeError: ignored

optimization

In [12]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

model = myModel()
model.cuda()

#학습을 위한 optimizer와 loss function 설정
optimizer = Adam(model.parameters(), lr=0.0001)
lf = CrossEntropyLoss().cuda()


#100번의 에폭을 실행
for e in range(100):
  print("\n\nepoch ", e)
  epoch_loss = 0
  train_correct = 0 
  
  #선언한 모델 오브젝트를 학습가능한 상태로 변경
  model.train()

  #모든 학습데이터에 대해서 학습
  for i in train_dataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)
    optimizer.zero_grad()
    data = i[0]
    data = data.cuda()
    target = i[1]
    target = target.reshape(-1).cuda()
    #print(target.shape)

    #결과 도출 및 정답수 연산
    output = model(data).reshape(-1,20)
    #print(output.shape)
    pred_label = torch.argmax(output, dim=-1)
    #print(pred_label.shape)
    #print(target.shape)
    train_correct += sum(pred_label == target)
    #print(sum(pred_label == target))
    #print(pred_label)
    #print(target)
    # print(data)

    #loss연산
    loss = lf(output, target.long())
    #print(loss)

    #loss backpropagation
    loss.backward()

    #gradient update
    optimizer.step()

    epoch_loss += loss.item()
  
  print(train_correct)
  print("train loss", epoch_loss/len(train_dataloader))
  print("train acc", train_correct/len(train_dataset))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0
  test_correct = 0 

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in test_dataloader:
      test_data = i[0]
      test_target = i[1]
      test_data = test_data.cuda()
      test_target = test_target.reshape(-1).cuda()
      #print(test_target.shape)

      test_output = model(test_data).reshape(-1,20)
      #print(test_output.shape)

      tloss = lf(test_output, test_target.long())
      test_pred_label = torch.argmax(test_output, dim=-1)
      #print(test_pred_label.shape)
      test_correct += sum(test_pred_label == test_target)
      test_loss += tloss.item()

  print("test loss", test_loss/len(test_dataloader))
  print("test acc", test_correct/len(test_dataset))
    




epoch  0
tensor(132093, device='cuda:0')
train loss 2.061704106628895
train acc tensor(33.0233, device='cuda:0')
test loss 1.2820453210310503
test acc tensor(40.3548, device='cuda:0')


epoch  1
tensor(178740, device='cuda:0')
train loss 0.8857589930295944
train acc tensor(44.6850, device='cuda:0')
test loss 0.4280779957771301
test acc tensor(46.5790, device='cuda:0')


epoch  2
tensor(186512, device='cuda:0')
train loss 0.3905206758528948
train acc tensor(46.6280, device='cuda:0')
test loss 0.3730955882505937
test acc tensor(46.5790, device='cuda:0')


epoch  3
tensor(186513, device='cuda:0')
train loss 0.3555018909275532
train acc tensor(46.6283, device='cuda:0')
test loss 0.35086891596967523
test acc tensor(46.5790, device='cuda:0')


epoch  4
tensor(186522, device='cuda:0')
train loss 0.3394892897456884
train acc tensor(46.6305, device='cuda:0')
test loss 0.3394970541650599
test acc tensor(46.5882, device='cuda:0')


epoch  5
tensor(186536, device='cuda:0')
train loss 0.330149652